In [1]:
import json
import numpy as np 
import pandas as pd
pd.options.display.max_columns = 999
pd.set_option('display.max_rows', 1500)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import re, string, ast

In [2]:
# Define functions to read massive (~7gb) json file
def init_ds(json):
    ds= {}
    keys = json.keys()
    for k in keys:
        ds[k]= []
    return ds, keys

def read_json(file):
    dataset = {}
    keys = []
    with open(file) as file_lines:
        for count, line in enumerate(file_lines):
            data = json.loads(line.strip())
            if count == 0:
                dataset, keys = init_ds(data)
            for k in keys:
                dataset[k].append(data[k])
                
        return pd.DataFrame(dataset)

# Handling Reviews JSON data

In [3]:
# Load the review json
df_review = read_json('../input/d/yelp-dataset/yelp-dataset/yelp_academic_dataset_review.json')

This on my machine took about ~10 gb ram

In [4]:
# Take a look at the review dataframe
df_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5.0,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [5]:
# Drop columns that are not necesary
df_review.drop(['review_id', 'user_id', 'useful', 'funny', 'cool', 'date'], axis=1, inplace=True)

### Review dataframe are used to narrow down businesses that are only restaurants.

In [6]:
# How many reviews are there?
print('There are total of {} reviews in the raw Yelp dataset'.format(len(df_review)))

There are total of 8635403 reviews in the raw Yelp dataset


In [7]:
# Filter reviews for text if it contains either the word 'food' or 'restaurant'
df_review = df_review[df_review['text'].str.contains('food|restaurant', flags=re.IGNORECASE, regex=True)]

In [8]:
# How many reviews are there afterwards?
print('There are total of {} reviews in the raw Yelp dataset that contain either food or restaurant in the review text'.format(len(df_review)))

There are total of 3419499 reviews in the raw Yelp dataset that contain either food or restaurant in the review text


In [9]:
df_review.head()

,business_id,stars,text
0,buF9druCkbuXLX526sGELQ,4.0,Apparently Prides Osteria had a rough summer a...
4,8zehGz9jnxPqXtOc7KaJxA,4.0,The food is always great here. The service fro...
6,EXOsmAB1s71WePlQk0WZrA,2.0,"The setting is perfectly adequate, and the foo..."
9,EEHhKSxUvJkoPSzeGKkpVg,5.0,I work in the Pru and this is the most afforda...
10,WQFn1A7-UAA4JT5YWiop_w,1.0,"They NEVER seem to get our \norder correct, se..."


In [10]:
# Get unique business ids
possible_restaurants = df_review['business_id'].unique()

# Handling Business JSON data

In [11]:
# Load the review json
df_business = read_json('../input/d/yelp-dataset/yelp-dataset/yelp_academic_dataset_business.json')

In [12]:
# Take a look at the business dataframe
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


##### For the attributes, different code will have to be executed to bring out the true values of the nested jsons. For the categories, since it's list of words separated by commas, NLP will be implemented.

In [13]:
# Re-load business.json in a list format
df_business = [json.loads(line) for line in open('../input/d/yelp-dataset/yelp-dataset/yelp_academic_dataset_business.json', 'r')]

# Normalize the df_business json
df_business = pd.json_normalize(df_business)

In [14]:
# Take a look at the un-nested business dataframe
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.RestaurantsTableService,attributes.WiFi,attributes.BikeParking,attributes.BusinessParking,attributes.BusinessAcceptsCreditCards,attributes.RestaurantsReservations,attributes.WheelchairAccessible,attributes.Caters,attributes.OutdoorSeating,attributes.RestaurantsGoodForGroups,attributes.HappyHour,attributes.BusinessAcceptsBitcoin,attributes.RestaurantsPriceRange2,attributes.Ambience,attributes.HasTV,attributes.Alcohol,attributes.GoodForMeal,attributes.DogsAllowed,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsAttire,attributes.RestaurantsDelivery,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday,attributes.GoodForKids,attributes.ByAppointmentOnly,hours,attributes.AcceptsInsurance,attributes,attributes.HairSpecializesIn,attributes.GoodForDancing,attributes.BestNights,attributes.Music,attributes.BYOB,attributes.CoatCheck,attributes.Smoking,attributes.DriveThru,attributes.BYOBCorkage,attributes.Corkage,attributes.RestaurantsCounterService,attributes.AgesAllowed,attributes.DietaryRestrictions,attributes.Open24Hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"Gastropubs, Food, Beer Gardens, Restaurants, B...",True,u'free',True,"{'garage': False, 'street': True, 'validated':...",True,False,True,True,True,True,True,False,2,"{'touristy': False, 'hipster': False, 'romanti...",True,'beer_and_wine',"{'dessert': False, 'latenight': False, 'lunch'...",False,True,u'average','casual',None,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",NaN,u'free',False,"{'garage': True, 'street': False, 'validated':...",True,False,NaN,True,False,False,NaN,NaN,2,"{'romantic': False, 'intimate': False, 'touris...",False,u'beer_and_wine',"{'dessert': False, 'latenight': False, 'lunch'...",NaN,True,u'average',u'casual',False,5:0-18:0,5:0-17:0,5:0-18:0,5:0-18:0,5:0-18:0,5:0-18:0,5:0-18:0,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"Antiques, Fashion, Used, Vintage & Consignment...",NaN,NaN,False,"{'garage': False, 'street': True, 'validated':...",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11:0-18:0,11:0-18:0,11:0-18:0,11:0-18:0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"Beauty & Spas, Hair Salons",NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"Gyms, Active Life, Interval Training Gyms, Fit...",NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16:0-19:0,16:0-19:0,16:0-19:0,16:0-19:0,16:0-19:0,9:0-11:0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As it can be seen some of the columns (eg. attributes.BusinessParking) still have nested JSON format

In [15]:
# How many businesses in the dataset?
print('There are {} in the business in the Yelp dataset'.format(len(df_business)))

There are 160585 in the business in the Yelp dataset


In [16]:
# First only keep necessary columns
attributes_list = [col for col in df_business.columns if 'attributes' in col]
keep_cols = ['is_open', 'business_id', 'name', 'stars', 'review_count', 'categories']
keep_cols.extend(attributes_list)
df_business = df_business[keep_cols].copy()

In [17]:
# Filter df_business to only contain business_id's that are possibly restaurants
df_business = df_business[df_business['business_id'].isin(possible_restaurants)]

# Filter df_business to only contain businesses where categories are under food, restaurant, and/or bar
df_business = df_business[df_business['categories'].str.contains('food|restaurant|bar', flags=re.IGNORECASE, na=False)]

df_business.reset_index(drop=True, inplace=True)

In [18]:
# How many restaurants in the dataset?
print('There are possibly {} in the restaurants in the Yelp dataset'.format(len(df_business)))

There are possibly 61328 in the restaurants in the Yelp dataset


In [19]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61328 entries, 0 to 61327
Data columns (total 46 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   is_open                                61328 non-null  int64  
 1   business_id                            61328 non-null  object 
 2   name                                   61328 non-null  object 
 3   stars                                  61328 non-null  float64
 4   review_count                           61328 non-null  int64  
 5   categories                             61328 non-null  object 
 6   attributes.RestaurantsTableService     19197 non-null  object 
 7   attributes.WiFi                        42389 non-null  object 
 8   attributes.BikeParking                 43103 non-null  object 
 9   attributes.BusinessParking             55834 non-null  object 
 10  attributes.BusinessAcceptsCreditCards  49757 non-null  object 
 11  at

In [20]:
# Fill attributes NaN value with "Not listed"
df_business[attributes_list] = df_business[attributes_list].fillna('Not listed')

#### The restaurant owners had the option to list or not list these attributes options.

In [21]:
# Search through the columns to find out which columns contain nested JSON format
nested_cols = []

for col in df_business.columns:
    if (df_business[col].dtypes == 'object') | (df_business[col].dtypes == 'str'):
        if any(df_business[col].str.contains('{', na=False)):
            nested_cols.append(col)

nested_cols

['attributes.BusinessParking',
 'attributes.Ambience',
 'attributes.GoodForMeal',
 'attributes.HairSpecializesIn',
 'attributes.BestNights',
 'attributes.Music',
 'attributes.DietaryRestrictions']

In [22]:
# Define a for loop fucntion to extract the information inside some of the attributes column

def attributes_extract(column):
    empty = "{'empty':'empty'}"
    result = pd.DataFrame()
    lst_dict = []
    global df_business
    
    # Create a loop to extract the information
    for row, value in df_business[column].items():
        if (value == 'Not listed') or (value == 'None'):
            temp_empty = ast.literal_eval(empty)
            lst_dict.append(temp_empty)

        else:
            temp_value = ast.literal_eval(value)
            lst_dict.append(temp_value)

    # Fill NaN and drop the 'empty' column
    result = result.append(lst_dict)
    result.fillna('Not listed', inplace=True)
    result.drop('empty', axis=1, inplace=True)

    # Rename the columns to include 'attributes' in the beginning
    for idx, col in enumerate(result.columns):
        result[col] = result[col].astype('str')
        result.rename(columns={col: ('attributes.' + col)}, inplace=True)

    # Drop the the column and concatenate extracted information to df_business dataframe
    df_business = pd.concat([df_business, result], axis=1)
    df_business.drop(column, axis=1, inplace=True)

In [23]:
# Un-nest the columns and add on to the business dataframe
for col in nested_cols:
    attributes_extract(col)

In [24]:
# Check to see if it worked correctly
df_business.head()

,is_open,business_id,name,stars,review_count,categories,attributes.RestaurantsTableService,attributes.WiFi,attributes.BikeParking,attributes.BusinessAcceptsCreditCards,attributes.RestaurantsReservations,attributes.WheelchairAccessible,attributes.Caters,attributes.OutdoorSeating,attributes.RestaurantsGoodForGroups,attributes.HappyHour,attributes.BusinessAcceptsBitcoin,attributes.RestaurantsPriceRange2,attributes.HasTV,attributes.Alcohol,attributes.DogsAllowed,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsAttire,attributes.RestaurantsDelivery,attributes.GoodForKids,attributes.ByAppointmentOnly,attributes.AcceptsInsurance,attributes,attributes.GoodForDancing,attributes.BYOB,attributes.CoatCheck,attributes.Smoking,attributes.DriveThru,attributes.BYOBCorkage,attributes.Corkage,attributes.RestaurantsCounterService,attributes.AgesAllowed,attributes.Open24Hours,attributes.garage,attributes.street,attributes.validated,attributes.lot,attributes.valet,attributes.touristy,attributes.hipster,attributes.romantic,attributes.divey,attributes.intimate,attributes.trendy,attributes.upscale,attributes.classy,attributes.casual,attributes.dessert,attributes.latenight,attributes.lunch,attributes.dinner,attributes.brunch,attributes.breakfast,attributes.straightperms,attributes.coloring,attributes.extensions,attributes.africanamerican,attributes.curly,attributes.kids,attributes.perms,attributes.asian,attributes.monday,attributes.tuesday,attributes.friday,attributes.wednesday,attributes.thursday,attributes.sunday,attributes.saturday,attributes.dj,attributes.background_music,attributes.no_music,attributes.jukebox,attributes.live,attributes.video,attributes.karaoke,attributes.dairy-free,attributes.gluten-free,attributes.vegan,attributes.kosher,attributes.halal,attributes.soy-free,attributes.vegetarian
0,1,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,4.0,86,"Gastropubs, Food, Beer Gardens, Restaurants, B...",True,u'free',True,True,False,True,True,True,True,True,False,2,True,'beer_and_wine',False,True,u'average','casual',None,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed
1,1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",Not listed,u'free',False,True,False,Not listed,True,False,False,Not listed,Not listed,2,False,u'beer_and_wine',Not listed,True,u'average',u'casual',False,True,False,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed
2,1,D4JtQNTI4X3KcbzacDJsMw,Bob Likes Thai Food,3.5,169,"Restaurants, Thai",Not listed,u'no',True,Not listed,True,Not listed,False,False,True,Not listed,Not listed,2,False,u'none',False,True,u'average',u'casual',True,True,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,Not listed,Not listed,Not li

In [25]:
# Reassign attributes_list
attributes_list = [col for col in df_business.columns if 'attributes' in col]

# Get list of columns where 'Not listed' (NaN) is >= 85% of the column values
dummy = []

for col in attributes_list:
    if df_business[col].value_counts(normalize=True)['Not listed'] >= 0.85:
        dummy.append(col)
dummy

['attributes.BusinessAcceptsBitcoin',
 'attributes.ByAppointmentOnly',
 'attributes.AcceptsInsurance',
 'attributes',
 'attributes.GoodForDancing',
 'attributes.BYOB',
 'attributes.CoatCheck',
 'attributes.Smoking',
 'attributes.DriveThru',
 'attributes.BYOBCorkage',
 'attributes.Corkage',
 'attributes.RestaurantsCounterService',
 'attributes.AgesAllowed',
 'attributes.Open24Hours',
 'attributes.straightperms',
 'attributes.coloring',
 'attributes.extensions',
 'attributes.africanamerican',
 'attributes.curly',
 'attributes.kids',
 'attributes.perms',
 'attributes.asian',
 'attributes.monday',
 'attributes.tuesday',
 'attributes.friday',
 'attributes.wednesday',
 'attributes.thursday',
 'attributes.sunday',
 'attributes.saturday',
 'attributes.dj',
 'attributes.background_music',
 'attributes.no_music',
 'attributes.jukebox',
 'attributes.live',
 'attributes.video',
 'attributes.karaoke',
 'attributes.dairy-free',
 'attributes.gluten-free',
 'attributes.vegan',
 'attributes.kosher',
 '

In [26]:
# Drop those columns as it could be introducing too much noise into the data
df_business.drop(dummy, axis=1, inplace=True)

In [27]:
df_business.head()

,is_open,business_id,name,stars,review_count,categories,attributes.RestaurantsTableService,attributes.WiFi,attributes.BikeParking,attributes.BusinessAcceptsCreditCards,attributes.RestaurantsReservations,attributes.WheelchairAccessible,attributes.Caters,attributes.OutdoorSeating,attributes.RestaurantsGoodForGroups,attributes.HappyHour,attributes.RestaurantsPriceRange2,attributes.HasTV,attributes.Alcohol,attributes.DogsAllowed,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsAttire,attributes.RestaurantsDelivery,attributes.GoodForKids,attributes.garage,attributes.street,attributes.validated,attributes.lot,attributes.valet,attributes.touristy,attributes.hipster,attributes.romantic,attributes.divey,attributes.intimate,attributes.trendy,attributes.upscale,attributes.classy,attributes.casual,attributes.dessert,attributes.latenight,attributes.lunch,attributes.dinner,attributes.brunch,attributes.breakfast
0,1,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,4.0,86,"Gastropubs, Food, Beer Gardens, Restaurants, B...",True,u'free',True,True,False,True,True,True,True,True,2,True,'beer_and_wine',False,True,u'average','casual',None,Not listed,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1,1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",Not listed,u'free',False,True,False,Not listed,True,False,False,Not listed,2,False,u'beer_and_wine',Not listed,True,u'average',u'casual',False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True
2,1,D4JtQNTI4X3KcbzacDJsMw,Bob Likes Thai Food,3.5,169,"Restaurants, Thai",Not listed,u'no',True,Not listed,True,Not listed,False,False,True,Not listed,2,False,u'none',False,True,u'average',u'casual',True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False
3,1,jFYIsSb7r1QeESVUnXPHBw,Boxwood Biscuit,4.5,11,"Breakfast & Brunch, Restaurants",Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed
4,1,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,4.0,39,"Food, Pizza, Restaurants",False,u'free',True,True,False,True,True,True,True,Not listed,2,True,u'none',True,True,'average',u'casual',True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed


In [28]:
# Check for unique values in all of attribute columns
attributes_list = [col for col in df_business.columns if 'attributes' in col]

for col in attributes_list:
    print(df_business[col].unique())   

['True' 'Not listed' 'False' 'None']
["u'free'" "u'no'" 'Not listed' "'free'" "'no'" "u'paid'" "'paid'" 'None']
['True' 'False' 'Not listed' 'None']
['True' 'Not listed' 'False' 'None']
['False' 'True' 'Not listed' 'None']
['True' 'Not listed' 'False' 'None']
['True' 'False' 'Not listed' 'None']
['True' 'False' 'Not listed' 'None']
['True' 'False' 'Not listed' 'None']
['True' 'Not listed' 'False' 'None']
['2' 'Not listed' '1' '3' '4' 'None']
['True' 'False' 'Not listed' 'None']
["'beer_and_wine'" "u'beer_and_wine'" "u'none'" 'Not listed' "'none'"
 "'full_bar'" "u'full_bar'" 'None']
['False' 'Not listed' 'True' 'None']
['True' 'Not listed' 'None' 'False']
["u'average'" 'Not listed' "'average'" "u'quiet'" "u'very_loud'" "'loud'"
 "u'loud'" "'quiet'" "'very_loud'" 'None']
["'casual'" "u'casual'" 'Not listed' "u'dressy'" "u'formal'" "'dressy'"
 'None' "'formal'"]
['None' 'False' 'True' 'Not listed']
['Not listed' 'True' 'False' 'None']
['False' 'True' 'Not listed']
['True' 'False' 'Not lis

#### It seems like there were some data issues with original Yelp data (adding u infront of some of the values)

In [29]:
# Which columns contain u' infront of the values?
dummy = []
for col in attributes_list:
    if any(df_business[col].str.contains("u'")):
        print(col)
        dummy.append(col)

attributes.WiFi
attributes.Alcohol
attributes.NoiseLevel
attributes.RestaurantsAttire


In [30]:
# Remove the character "u'" infront of the strings
for col in dummy:
    df_business[col] = df_business[col].str.replace("u'", "'")

The reason for replacing u' with ' is because values like u'loud will become lod if only replaced with u

In [31]:
# Check to see if the above worked
for col in dummy:
    print(df_business[col].unique())   

["'free'" "'no'" 'Not listed' "'paid'" 'None']
["'beer_and_wine'" "'none'" 'Not listed' "'full_bar'" 'None']
["'average'" 'Not listed' "'quiet'" "'very_loud'" "'loud'" 'None']
["'casual'" 'Not listed' "'dressy'" "'formal'" 'None']


##### As shown above, it seems like original data had some of the NaN values be placed with 'None'. Previously, we've replaced all NaN with 'Not listed'. Replace these false 'None' with 'Not listed'

In [32]:
# Find out which columns have 'None'
none_cols = []

for col in attributes_list:
    if df_business[col].str.contains('None', flags=re.IGNORECASE).any():
        none_cols.append(col)

none_cols

['attributes.RestaurantsTableService',
 'attributes.WiFi',
 'attributes.BikeParking',
 'attributes.BusinessAcceptsCreditCards',
 'attributes.RestaurantsReservations',
 'attributes.WheelchairAccessible',
 'attributes.Caters',
 'attributes.OutdoorSeating',
 'attributes.RestaurantsGoodForGroups',
 'attributes.HappyHour',
 'attributes.RestaurantsPriceRange2',
 'attributes.HasTV',
 'attributes.Alcohol',
 'attributes.DogsAllowed',
 'attributes.RestaurantsTakeOut',
 'attributes.NoiseLevel',
 'attributes.RestaurantsAttire',
 'attributes.RestaurantsDelivery',
 'attributes.GoodForKids']

In [33]:
# Take a look at one example column from above
df_business['attributes.RestaurantsTableService'].unique()

array(['True', 'Not listed', 'False', 'None'], dtype=object)

In [34]:
# Replace 'None' with 'Not listed'
for col in none_cols:
    df_business[col] = df_business[col].str.replace('None', 'Not listed', flags=re.IGNORECASE)

In [35]:
# Re-look at that example column to see if 'None' has been succesfully replaced
df_business['attributes.RestaurantsTableService'].unique()

array(['True', 'Not listed', 'False'], dtype=object)

In [36]:
# Check the dataframe
df_business.head()

,is_open,business_id,name,stars,review_count,categories,attributes.RestaurantsTableService,attributes.WiFi,attributes.BikeParking,attributes.BusinessAcceptsCreditCards,attributes.RestaurantsReservations,attributes.WheelchairAccessible,attributes.Caters,attributes.OutdoorSeating,attributes.RestaurantsGoodForGroups,attributes.HappyHour,attributes.RestaurantsPriceRange2,attributes.HasTV,attributes.Alcohol,attributes.DogsAllowed,attributes.RestaurantsTakeOut,attributes.NoiseLevel,attributes.RestaurantsAttire,attributes.RestaurantsDelivery,attributes.GoodForKids,attributes.garage,attributes.street,attributes.validated,attributes.lot,attributes.valet,attributes.touristy,attributes.hipster,attributes.romantic,attributes.divey,attributes.intimate,attributes.trendy,attributes.upscale,attributes.classy,attributes.casual,attributes.dessert,attributes.latenight,attributes.lunch,attributes.dinner,attributes.brunch,attributes.breakfast
0,1,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,4.0,86,"Gastropubs, Food, Beer Gardens, Restaurants, B...",True,'free',True,True,False,True,True,True,True,True,2,True,'beer_and_wine',False,True,'average','casual',Not listed,Not listed,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1,1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,4.0,126,"Salad, Soup, Sandwiches, Delis, Restaurants, C...",Not listed,'free',False,True,False,Not listed,True,False,False,Not listed,2,False,'beer_and_wine',Not listed,True,'average','casual',False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True
2,1,D4JtQNTI4X3KcbzacDJsMw,Bob Likes Thai Food,3.5,169,"Restaurants, Thai",Not listed,'no',True,Not listed,True,Not listed,False,False,True,Not listed,2,False,'Not listed',False,True,'average','casual',True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False
3,1,jFYIsSb7r1QeESVUnXPHBw,Boxwood Biscuit,4.5,11,"Breakfast & Brunch, Restaurants",Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed
4,1,HPA_qyMEddpAEtFof02ixg,Mr G's Pizza & Subs,4.0,39,"Food, Pizza, Restaurants",False,'free',True,True,False,True,True,True,True,Not listed,2,True,'Not listed',True,True,'average','casual',True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,Not listed,Not listed,Not listed,Not listed,Not listed,Not listed


In [37]:
df_business.shape

(61328, 45)

In [38]:
# Create a csv file which consists of mostly the restaurants
df_business.to_csv('yelp_restaurants.csv', encoding='utf-8', index=False)